# Image Processing with ITK

This notebook shows using Dask to apply ITK functions many times over large volumes of data.

As of 2019-12-24 it requires a pre-release of ITK

In [ ]:
!pip install --pre itk

## Start Dask Client for Dashboard

Starting the Dask Client will provide a dashboard which 
is useful to gain insight on the computation.

In [ ]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1, processes=False)
client

## Stack of images

We can apply ITK functions along a stack of images with the `apply_gufunc` function.

In [ ]:
import dask.array as da

x = da.random.random((1000, 500, 500), chunks=("auto", None, None))
x

In [ ]:
import itk

y = da.apply_gufunc(itk.median_image_filter, "(i,j)->(i,j)", x, radius=2)
y

In [ ]:
y.max().compute()

## Single large image

Or we can apply them with a small halo around them with the `map_overlap` function

In [ ]:
import dask.array as da, itk

x = da.random.random((100000, 100000))
x

In [ ]:
z = x.map_overlap(itk.median_image_filter, radius=2, depth=2)
z

In [ ]:
z[:10000, :10000].sum().compute()

## Memory issues

On my system memory seems to hang around for a while when I use a small chunk size and multiple threads.  I'm not sure why.  Dask has removed all of its references to things.

I recommend restarting things before running this section

In [ ]:
import dask
dask.config.set({"array.chunk-size": "32 MiB"})

In [ ]:
import psutil
from dask.utils import format_bytes

proc = psutil.Process()

In [ ]:
import dask.array as da, itk

x = da.random.random((100000, 100000))
z = x.map_overlap(itk.median_image_filter, radius=2, depth=2)

In [ ]:
print("Before:", format_bytes(proc.memory_info().rss))
z[:10000, :10000].compute(scheduler="single-threaded")
print("After:", format_bytes(proc.memory_info().rss))

In [ ]:
print("Before:", format_bytes(proc.memory_info().rss))
z[:10000, :10000].compute(scheduler="threading")
print("After:", format_bytes(proc.memory_info().rss))